# Red Neuronal Profunda


**Notacion**:
- Superscript $[l]$ denota la cantidad  asociada con la capa $l^{th}$ . 
    - Ejemplo: $a^{[L]}$ es la $L^{th}$ capa de activacion. $W^{[L]}$ y $b^{[L]}$ son los parametros de la capa $L^{th}$ .
- Superscript $(i)$ denota la cantidad asociada con el  $i^{th}$ ejemplo. 
    - Ejemplo: $x^{(i)}$ es el $i^{th}$ ejemplor de entrenamiento.
- Lowerscript $i$ denota la $i^{th}$ entrada del vector.
    - Ejemplo: $a^{[l]}_i$ denota ls $i^{th}$ entrada a la capa de acticacion $l^{th}$ 


## 1 - Importar modulos


In [0]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer


% matplotlib inline


### Breast Cancer Dataset 

In [49]:
cancer=load_breast_cancer()
X=cancer.data
Y=cancer.target

from sklearn.model_selection import train_test_split

X_train,X_test, y_train,y_test= train_test_split(X,Y,test_size=0.2, shuffle =True)

print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')

y_train=y_train.reshape(-1,1)
y_test=y_test.reshape(-1,1)

print(f'y_train shape: {y_train.shape}')
print(f'y_test shape: {y_test.shape}')

y_train=y_train.reshape(-1,1)


X_train shape: (455, 30)
X_test shape: (114, 30)
y_train shape: (455, 1)
y_test shape: (114, 1)


### Mnist Dataset

In [0]:
from tensorflow.keras.datasets import mnist
(X_train,y_train),(X_test, y_test) = mnist.load_data()

X_train= X_train.reshape(-1, 28*28)
y_train=y_train.reshape(-1,1)

def one_hot_encoding(Y):
  n_classes=len(np.unique(Y))
  encoding= np.eye(n_classes)[Y.reshape(-1)]
  return encoding

Y= one_hot_encoding(y_train)

## 2 - Inicializacion


### 2.1 - L-layer Neural Network

construiremos una funcion para $L$ capas 
 Recuerda que  $n^{[l]}$ es el  numero de unidades de la capa  $l$. Por ejepmplo si el tamaño de la entrada es  $X$ con forma $(12288, 209)$ (donde $m=209$ ejemplos) entonces:

<table style="width:100%">

    <tr>
        <td>  </td> 
        <td> **Shape of W** </td> 
        <td> **Shape of b**  </td> 
        <td> **Activation** </td>
        <td> **Shape of Activation** </td> 
    </tr>

    <tr>
        <td> **Layer 1** </td> 
        <td> $(n^{[1]},12288)$ </td> 
        <td> $(n^{[1]},1)$ </td> 
        <td> $Z^{[1]} = W^{[1]}  X + b^{[1]} $ </td> 
        <td> $(n^{[1]},209)$ </td> 
    </tr>

    <tr>
        <td> **Layer 2** </td> 
        <td> $(n^{[2]}, n^{[1]})$  </td> 
        <td> $(n^{[2]},1)$ </td> 
        <td>$Z^{[2]} = W^{[2]} A^{[1]} + b^{[2]}$ </td> 
        <td> $(n^{[2]}, 209)$ </td> 
    </tr>

    <tr>
        <td> $\vdots$ </td> 
        <td> $\vdots$  </td> 
        <td> $\vdots$  </td> 
        <td> $\vdots$</td> 
        <td> $\vdots$  </td> 
    </tr>

    <tr>
        <td> **Layer L-1** </td> 
        <td> $(n^{[L-1]}, n^{[L-2]})$ </td> 
        <td> $(n^{[L-1]}, 1)$  </td> 
        <td>$Z^{[L-1]} =  W^{[L-1]} A^{[L-2]} + b^{[L-1]}$ </td> 
        <td> $(n^{[L-1]}, 209)$ </td> 
    </tr>


    <tr>
        <td> **Layer L** </td> 
        <td> $(n^{[L]}, n^{[L-1]})$ </td> 
        <td> $(n^{[L]}, 1)$ </td>
        <td> $Z^{[L]} =  W^{[L]} A^{[L-1]} + b^{[L]}$</td>
        <td> $(n^{[L]}, 209)$  </td> 
    </tr>

</table>

Recuerda que cuando computas  $W X + b$ en python, hace el broadcasting. Por ejemplo, si: 

$$ W = \begin{bmatrix}
    j  & k  & l\\
    m  & n & o \\
    p  & q & r 
\end{bmatrix}\;\;\; X = \begin{bmatrix}
    a  & b  & c\\
    d  & e & f \\
    g  & h & i 
\end{bmatrix} \;\;\; b =\begin{bmatrix}
    s  \\
    t  \\
    u
\end{bmatrix}\tag{2}$$

Entonces  $WX + b$ sera:

$$ WX + b = \begin{bmatrix}
    (ja + kd + lg) + s  & (jb + ke + lh) + s  & (jc + kf + li)+ s\\
    (ma + nd + og) + t & (mb + ne + oh) + t & (mc + nf + oi) + t\\
    (pa + qd + rg) + u & (pb + qe + rh) + u & (pc + qf + ri)+ u
\end{bmatrix}\tag{3}  $$


**Instruciones**:
- La estructura del modelo es:   *[LINEAR -> RELU] $ \times$ (L-1) -> LINEAR -> SIGMOID*. I.e., que tiene  $L-1$ capas usando a ReLU activation seguida de un sigmoide
- Usa random initialization para las matrices weight. 
- Usa zeros initialization para los  biases
- Nosotros guardaremos el valor  $n^{[l]}$, que es el numero de las unidades en las diferentes capas , en  la variable `layer_dims`. Por example, una `layer_dims` de forma [2,4,1]: Eso significa que tenemos dos entradas, una hidden layer con 4 hidden units, y una output layer una  unidad .

In [0]:
def initialize_parameters_deep(layer_dims):
    """
    Argumentos:
    layer_dims -- python array (list) que contiene las dimensiones de nuestra red neuronal
    p.e [In, 100, 100,50,10]
    
    Returns:
    parameters -- python diccionario que contiene los parametros "W1", "b1", ..., "WL", "bL":
                    Wl -- weight matrix de la forma (layer_dims[l], layer_dims[l-1])
                    bl -- bias vector de la forma (layer_dims[l], 1)
    """
    np.random.seed(3)
    parametros={}
    L =len(layer_dims) # numero de capas
    
    for l in range(1, L):
        parametros['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1])*0.01 # Forma W: (N[l], N[l-1])
        parametros['b' + str(l)] = np.zeros((layer_dims[l],1)) # Forma Bias : (N[l], 1)
        
    return parametros

In [4]:
param= initialize_parameters_deep([5,4,3])
param

{'W1': array([[ 1.78862847,  0.43650985,  0.09649747, -1.8634927 , -0.2773882 ],
        [-0.35475898, -0.08274148, -0.62700068, -0.04381817, -0.47721803],
        [-1.31386475,  0.88462238,  0.88131804,  1.70957306,  0.05003364],
        [-0.40467741, -0.54535995, -1.54647732,  0.98236743, -1.10106763]]),
 'W2': array([[-1.18504653, -0.2056499 ,  1.48614836,  0.23671627],
        [-1.02378514, -0.7129932 ,  0.62524497, -0.16051336],
        [-0.76883635, -0.23003072,  0.74505627,  1.97611078]]),
 'b1': array([[0.],
        [0.],
        [0.],
        [0.]]),
 'b2': array([[0.],
        [0.],
        [0.]])}

## 3 - Forward propagation modulo

### 3.1 - Linear Forward 
Modulo Linear Foward:

- LINEAR
- LINEAR -> ACTIVATION donde  ACTIVATION puede ser ReLU o Sigmoid. 
- [LINEAR -> RELU] $\times$ (L-1) -> LINEAR -> SIGMOID 

La funcion vectorizada para computar el Foward prop modulo sobre todos los ejemplos es:

$$Z^{[l]} = W^{[l]}A^{[l-1]} +b^{[l]}\tag{4}$$

donde $A^{[0]} = X$. 


In [0]:
def linear_forward(A, W, b):
    """
    Implementa el modulo de forward propagation para Lth capas.

    Argumentos:
    A -- activaciones de la capa anterior(o datos de entrada): (Tamaño de la capa anterior, numero de ejemplos)
    W -- weights matrix: numpy array de la forma (Tamaño de la capa actual, Tamaño de la capa anterior)
    b -- bias vector, numpy array de la forma (Tamaño de la capa actual, 1)

    Returns:
    Z -- la entrada de la activacion 
    cache --  "A", "W" y "b" 
    """
    # Transformacion lineal
    Z =W @ A + b
    
    cache=(A,W,b)
    
    return Z, cache

In [0]:
param= initialize_parameters_deep([784,4,3])

### 3.2 - Linear-Activation Forward

Funciones de activacion

- **Sigmoid**: $\sigma(Z) = \sigma(W A + b) = \frac{1}{ 1 + e^{-(W A + b)}}$.
``` python
A, activation_cache = sigmoid(Z)
```

- **ReLU**: La formula matematica  para ReLu es $A = RELU(Z) = max(0, Z)$. 
``` python
A, activation_cache = relu(Z)
```

Implementar el paso de  forward propagation para la capa  *LINEAR->ACTIVATION* . 

La relacion matematica es la siguiente: $A^{[l]} = g(Z^{[l]}) = g(W^{[l]}A^{[l-1]} +b^{[l]})$ donde   "g"  es la fn activacion que puede ser ya sea sigmoid() o relu(). 

In [0]:
def sigmoid(Z):
  return 1.0/(1 + np.exp(-np.clip(Z, -250, 250)))

def relu(Z):
    """Funcion de activacion ReLu"""
    A=np.maximum(0,Z)
    return A

In [0]:

def linear_activation_forward(A_prev, W, b, activation):
    """
    Implementar el paso de  forward propagation para la capa LINEAR->ACTIVATION 

    Argumentos:
    A_prev -- activaciones de la capa anterior(o datos de entrada): (Tamaño de la capa anterior, numero de ejemplos)
    W -- weights matrix: numpy array de la forma (Tamaño de la capa actual, Tamaño de la capa anterior)
    b -- bias vector, numpy array de la forma (Tamaño de la capa actual, 1)
    activation -- la fn de  activacion que sera usada en esta capa
    
    
    Returns:
    A -- post-activacion 
    cache --  diccionario "linear_cache" y "activation_cache";
             
    """
    Z, cache= linear_forward(A_prev, W, b)
    
    if activation == "sigmoid":
        A = sigmoid(Z)
        
    else:
        A = relu(Z) # Rectifier Linear Unit

    return A, cache

### 3.3 Modelo de L capas



In [0]:
def L_model_forward(X, parametros):
    """
    Implementa forward propagation para [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID 
    
    Arguments:
    X -- datos, numpy array de shape (Tamaño de entrada, numero de ejemplos)
    parametros -- salida de  initialize_parameters_deep()
    
    Returns:
    AL --  valor de post-activacion 
    caches 
    """
    caches=[]
    A=X# X=  A^[0] primera capa
    L= len(parametros)//2  # numero de capas o layers
    
    # Linear --> Relu
    for l in range(1,L):
        # X
        #activation= 'relu ' if l< L else 'sigmoid'
        A, cache =linear_activation_forward(A, parametros['W' + str(l)], parametros['b' + str(l)],activation='relu')
        # guarda Z ,A , etc que nos sirve para el backward prop
        caches.append(cache)

    AL, cache = linear_activation_forward(A, parametros["W" + str(L)], parametros["b"+str(L)], activation= 'sigmoid')
    caches.append(cache)
    return AL, caches

## 4 - Cost function


**Crea la funcion**: Categorical cross-entropy cost 
$$J=-\frac{1}{m} \sum\limits_{i = 1}^{m} (y^{(i)}\log\left(a^{[L] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right)) \tag{7}$$


In [0]:
def compute_cost(AL, Y):
    """
    Implementa la cost function  (7).

    Argumentoss:
    AL -- vector de probabilidad correspondinte a las predicciones, shape (1, numero de ejemplos)
    Y -- vector de  clases o categorias a predecir, shape (1, numero de ejemplos)

    Returns:
    cost -- cross-entropy cost
    """
    eps = np.finfo(float).eps
    m= Y.shape[0]
    loss= -np.sum(Y @ np.log(AL.T + eps) + (1-Y) @ np.log(1-AL.T + eps))/m
    
    return np.squeeze(loss) 

## 5 - Backward propagation modulo



**Recordatorio sobre backprop** 
Recuerda que para encontrar los gradientes tienes que computar la derivada parcial $\mathcal{L}$ con respecto a los parametros por ejemplo $z^{[1]}$ para una rnn de dos capas:

$$\frac{d \mathcal{L}(a^{[2]},y)}{{dz^{[1]}}} = \frac{d\mathcal{L}(a^{[2]},y)}{{da^{[2]}}}\frac{{da^{[2]}}}{{dz^{[2]}}}\frac{{dz^{[2]}}}{{da^{[1]}}}\frac{{da^{[1]}}}{{dz^{[1]}}} \tag{8} $$



Ahora computaremos el modulo de backprop en tres pasos:
- LINEAR backward
- LINEAR -> ACTIVATION backward donde ACTIVATION computa la derivada de  ReLU o sigmoid 
- [LINEAR -> RELU] $\times$ (L-1) -> LINEAR -> SIGMOID backward 

### 5.1 - Linear backward

Formulas vectorizada para el modulo de bp

$$ dW^{[l]} = \frac{\partial \mathcal{L} }{\partial W^{[l]}} = \frac{1}{m} dZ^{[l]} A^{[l-1] T} \tag{8}$$
$$ db^{[l]} = \frac{\partial \mathcal{L} }{\partial b^{[l]}} = \frac{1}{m} \sum_{i = 1}^{m} dZ^{[l](i)}\tag{9}$$
$$ dA^{[l-1]} = \frac{\partial \mathcal{L} }{\partial A^{[l-1]}} = W^{[l] T} dZ^{[l]} \tag{10}$$


In [0]:
def linear_backward(dZ, cache):
    """
    Implementa el modulo para  backward propagation para una capa

    Argumentos:
    dZ -- Gradiente del cost con respecto a la capa actual 
    cache -- tupla de los valores (A_prev, W, b) proveninetes del  forward propagation en la capa actual

    Returns:
    dA_prev -- Gradiente del cost con respecto a la activacion
    dW -- Gradiente del  cost con respecto a W (capa actual l)
    db -- Gradiente del  cost con respecto a b (capa actual l)
    """
    A_prev,W,b = cache
    m =A_prev.shape[0]
    
    dW =(dZ @ A_prev.T)/m # misma forma que W
    db= np.sum(dZ,axis=1, keepdims=True)/m # misma forma que b
    dA_prev =W.T @ dZ # misma forma que A_prev
    
    return dA_prev, dW, db

### 5.2 - Linear-Activation backward



Si $g(.)$ es la  fn activacion 
`sigmoid_backward` y `relu_backward` computan  la derivada de la fn actvivacion $$dZ^{[l]} = dA^{[l]} * g'(Z^{[l]}) \tag{11}$$.  

Implementa la capa de  backpropagation  *LINEAR->ACTIVATION* .

In [0]:
def linear_activation_backward(dA, cache, activation):
    """
    Implementa la capa de backward propagation para LINEAR->ACTIVATION
    
    Argumentos:
    dA -- gradiente de post-activacion de l capa actual l 
    cache -- tuple de valores (linear_cache, activation_cache) 
    activation -- fn activacion
    
    Returns:
    dA_prev -- Gradient del cost con respecto a la fn de activacion (de la capa anterior l-1)
    dW -- Gradiente del  cost con respecto a W (capa actual l)
    db -- Gradiente del  cost con respecto a b (capa actual l)
    """
    # Derivada de la funcion de acticacion Relu
    if activation =='relu':
        # si X>0 ponlo 1 sino 0
        dZ = (dA>0)
        
    # Derivada de la funcion de acticacion sigmoide
    elif activation =='sigmoid':
        # s=s*(1-s)
        s= sigmoid(dA) # sigmoid 
        dZ= dA*s*(1-s)
    
    dA_prev,dW,db=linear_backward(dZ,cache)
    
    return dA_prev, dW, db

### 5.3 - L-Modelo Backward 


** Inicializacion de backpropagation**:
Para hacer backprop atraves de la rednn, sabemos que la salida es, 
$A^{[L]} = \sigma(Z^{[L]})$. Y la derivada de su funcion de error con tespecto a A  es : $dA^{[L]}= \frac{\partial \mathcal{L}}{\partial A^{[L]}}$.

donde $dA^{[L]}= \frac YA + \frac{(1-Y)}{(1-A)}$

una forma de hacerlo con numpy es :

```python
dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL)) 
```


In [0]:
def L_model_backward(AL, Y, caches):
    """
    Implementa el modulo de backward propagation para  [LINEAR->RELU] * (L-1) -> LINEAR -> SIGMOID 
    
    Argumentos:
    AL -- vector de probabilidad, salida del forward propagation (L_model_forward())
    Y -- clases 
    caches -- 
    
    Returns:
    grads -- Un diccionario de los gradientes
             grads["dA" + str(l)] = ... 
             grads["dW" + str(l)] = ...
             grads["db" + str(l)] = ... 
    """
    grads={}
    L=len(caches) # numero de capas
    m =AL.shape[0]
    eps = np.finfo(float).eps
    
    # Derivada de Loss con respecto a A 
    dAL =  - (np.divide(Y, AL + eps ) - np.divide(1 - Y, 1 - AL+ eps))
    
    # Inicializar Gradientes
    # Backprop Sigmoide --> linear
    grads['dA'+ str(L)], grads['dW'+str(L)], grads['db'+ str(L)] = linear_activation_backward(dAL,caches[L-1],activation='sigmoid')
    
    # Backprop Relu ---> Linear
    for l in reversed(range(L-1)):
       # current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads['dA' + str(l+2)], caches[l], activation = "relu")
        grads["dA" + str(l + 1)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp
    

    return grads

## 6  - Algoritmo de optimizacion 

Gradient descent: 

$$ W^{[l]} = W^{[l]} - \alpha \text{ } dW^{[l]} \tag{16}$$
$$ b^{[l]} = b^{[l]} - \alpha \text{ } db^{[l]} \tag{17}$$

donde  $\alpha$ es el  learning rate. 

In [0]:
def update_parameters(parametros, grads, learning_rate):
    """
    Gradient descent
    
    Argumentos:
    parametros  
    grads  
    
    Returns:
    parametros -- actualizados 
    """
    # Numero de capas
    L = len(parametros)//2 

    for l in range(1,L):
      parametros['W' + str(l)] += - learning_rate*grads['dW' + str(l)]
      parametros['b' + str(l)] += - learning_rate*grads['db' + str(l)]
    
    return parameters

## Aplication

In [0]:

def L_layer_model(X, Y, layers_dims, learning_rate=0.0075, num_iterations=3000, print_cost=False): 

    np.random.seed(1)
    costs = []       
    
    # Parameters initialization.
    parameters = initialize_parameters_deep(layers_dims)
    
    # Loop (gradient descent)
    for i in range(0, num_iterations):

        # Forward propagation: [LINEAR -> RELU]*(L-1) -> LINEAR -> SIGMOID.
        AL, caches = L_model_forward(X, parameters)
        
        # Compute cost.
        cost = compute_cost(AL, Y)
        
        # Backward propagation.
        grads = L_model_backward(AL, Y, caches)
        
        # Update parametros
        parameters = update_parameters(parameters, grads, learning_rate)
                
        # Print the cost every 100 training example
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" % (i, cost))
        if print_cost and i % 100 == 0:
            costs.append(cost)
            
    # plot the cost
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per tens)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    return parameters

In [0]:
layers_dims = [784, 20, 7, 5, 10] #  5-layer model
parametros= L_layer_model(X_train.T, Y.T, layers_dims,learning_rate=0.01, num_iterations=2500, print_cost=True)

## 7 Prediccion


In [0]:
def predict(X, y, parametros):
    """
    .
    
    Argumentos:
    X -- data set 
    parametros
    
    Returns:
    p -- predicciones del dataset dado X
    """
    
    
        
    return p